# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [4]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
#Your code here


In [6]:
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [7]:
df['count'] = 1
df

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
...,...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view,1
8184,2017-01-18 09:42:12.844575,755912,experiment,view,1
8185,2017-01-18 10:01:09.026482,458115,experiment,view,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1


In [10]:
control = df[df["group"] == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)
control

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [9]:
experiment = df[df["group"] == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)
experiment

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [11]:
control["click"].sum()

932.0

In [12]:
experiment["click"].sum()

928.0

In [13]:
import numpy as np
import scipy.stats as stats

In [15]:
tnum = control["click"].mean() - experiment["click"].mean()
    
tden = np.sqrt(control["click"].var(ddof=1)/control["click"].size + experiment["click"].var(ddof=1)/experiment["click"].size)
    
welchtvalue = np.abs(tnum/tden)
welchtvalue

2.615440020788212

In [16]:
s1 = control["click"].var(ddof=1) 
s2 = experiment["click"].var(ddof=1)
n1 = control["click"].size
n2 = experiment["click"].size
    
dofnum = (s1/n1 + s2/n2)**2
dofden = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)

dof = dofnum/dofden
dof

6211.149291520598

In [17]:
p = 1-stats.t.cdf(np.abs(welchtvalue), dof)
p

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [18]:
#Your code here
control["click"].mean() * len(experiment)

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [19]:
#Your code here
n = len(experiment)
p = control["click"].mean()

In [20]:
var = n * p * (1-p)
var

603.6135462588397

In [21]:
std = np.sqrt(var)
std

24.568547907005815

In [22]:
z = (experiment["click"].sum() - (control["click"].mean() * len(experiment)))/std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [23]:
#Your code here
p_value = stats.norm.sf(z)
p_value


0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The first p we calculated was ~.004. The second verification p value we calculated was even lower ~.0001.
Both are low enough that we can say the experimental looks like it works, and produces more clicks.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.